# Cestovanie za ustavnou zdravotnou starostlivostou v roku 2023

## Imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json
import re

import seaborn as sns
import os

## Data imports

v1: predbezne vysledky po errate, ulozene na SharePointe OSN. Este sa budu updatovat

In [8]:
all_inpatients_file_path = r"C:\Users\hunakj\health.gov.sk\IZA - Cestovanie za ZS\2023\Data inputs\osn_vsetka_starostlivost_2023_ms_v2024.2.csv"

In [10]:
all_inpatients = pd.read_csv(all_inpatients_file_path, dtype=str, delimiter=";")

In [11]:
all_inpatients.head()

,id_hp,vek_roky,vek_dni,hmotnost,upv,diagnozy,vykony,drg,erv,typ_starostlivosti,diagnozy_z_01,vykony_z_01,obdobie_od,obdobie_do,datum_od,datum_do,id_poistenca,datum_narodenia,kod_pobytu,kod_zp,pzs_12,IDENTIFZAR,pzs_8,pzs_6,id_hp_pzs,ms
0,35380_23000079,20,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-16,5130405,2002-04-25,522279,24,N61173014101,N6117301,N6117301,N61173,23000079,S05-34~S05-29
1,35380_23000214,19,NaN,NaN,NaN,j350,5g603&B&20230127,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-26,2023-01-31,2778465,2003-08-23,527696,24,N61173014101,N6117301,N6117301,N61173,23000214,S05-34~S05-29
2,35380_23000126,19,NaN,NaN,NaN,j350~j352,5g603&B&20230118~5g6071&Z&20230118,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-17,2023-01-21,3305786,2003-10-07,529249,24,N61173014101,N6117301,N6117301,N61173,23000126,S05-34~S05-29
3,35380_23000083,18,NaN,NaN,NaN,j350,5g603&B&20230113,d30b,0.8431,DRG,j350,NaN,2023-01,2023-01,2023-01-12,2023-01-17,3305359,2004-08-15,520128,24,N61173014101,N6117301,N6117301,N61173,23000083,S53-33~S63-40
4,35380_23000159,18,NaN,NaN,NaN,h912,NaN,d61a,0.7048,DRG,h912,NaN,2023-01,2023-01,2023-01-20,2023-01-26,2852825,2004-08-31,520802,24,N61173014101,N6117301,N6117301,N61173,23000159,S53-29


In [12]:
all_inpatients.dtypes

id_hp                 object
vek_roky              object
vek_dni               object
hmotnost              object
upv                   object
diagnozy              object
vykony                object
drg                   object
erv                   object
typ_starostlivosti    object
diagnozy_z_01         object
vykony_z_01           object
obdobie_od            object
obdobie_do            object
datum_od              object
datum_do              object
id_poistenca          object
datum_narodenia       object
kod_pobytu            object
kod_zp                object
pzs_12                object
IDENTIFZAR            object
pzs_8                 object
pzs_6                 object
id_hp_pzs             object
ms                    object
dtype: object

In [13]:
all_inpatients.shape

(1178065, 26)

zmena